### Set up a Python virtual environment in Visual Studio Code

1. Open the Command Palette (Ctrl+Shift+P).
1. Search for **Python: Create Environment**.
1. Select **Venv**.
1. Select a Python interpreter. Choose 3.10 or later.

It can take a minute to set up. If you run into problems, see [Python environments in VS Code](https://code.visualstudio.com/docs/python/environments).

### Install packages

In [1]:
! pip install -r requirements.txt --quiet

In [2]:
# LangChain Imports needed
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.retrievers import (
    AzureCognitiveSearchRetriever,
)
from langchain_openai import AzureOpenAIEmbeddings

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [3]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
import os

load_dotenv(override=True) # take environment variables from .env.

# The following variables from your .env file are used in this notebook
endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
azure_search_name = os.environ["AZURE_SEARCH_SERVICE_NAME"]
azure_search_admin_key = os.environ["AZURE_SEARCH_ADMIN_KEY"]
credential = AzureKeyCredential(azure_search_admin_key) if len(azure_search_admin_key) > 0 else DefaultAzureCredential()
index_name = os.environ["AZURE_SEARCH_INDEX"]
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_key = os.environ["AZURE_OPENAI_KEY"] if len(os.environ["AZURE_OPENAI_KEY"]) > 0 else None
azure_openai_embedding_deployment = os.environ["AZURE_OPENAI_EMBEDDING_DEPLOYMENT"]
embedding_model_name = os.environ["AZURE_OPENAI_EMBEDDING_MODEL_NAME"]
azure_openai_api_version = os.environ["AZURE_OPENAI_API_VERSION"]

azure_openai_model_deployment = os.environ["AZURE_OPENAI_MODEL_DEPLOYMENT"]

## Setting up the client configurations for Azure OpenAI and AI Search

In [4]:
COMPLETION_TOKENS = 2500
llm = AzureChatOpenAI(deployment_name=azure_openai_model_deployment, 
                      temperature=0, 
                      max_tokens=COMPLETION_TOKENS,
                      openai_api_version=azure_openai_api_version,
                      openai_api_key=azure_openai_key)

embeddings = AzureOpenAIEmbeddings(
    deployment=azure_openai_embedding_deployment,
    model=embedding_model_name,
    azure_endpoint=azure_openai_endpoint,
    openai_api_key=azure_openai_key,
    openai_api_version=azure_openai_api_version
)

retriever = AzureCognitiveSearchRetriever(index_name=index_name, service_name=azure_search_name, api_key=azure_search_admin_key, content_key="chunk", top_k=3)


## Defining the prompt template and the RAG pipeline

In [5]:
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)


In [6]:
question = "What is Chain-of-Thought Prompting?"

response = retrieval_chain.invoke({"input": question})
print(response["answer"])

Chain-of-Thought Prompting is a mechanism used to elicit multi-step reasoning behavior in large language models. It involves prompting an off-the-shelf language model with a chain of thought, which improves performance on reasoning tasks and facilitates length generalization beyond seen chains of thought.


## Asking a questions out of the context of the documents ingested

In [8]:
question = "Who was the first man to climb everest?"

response = retrieval_chain.invoke({"input": question})
print(response["answer"])

The question cannot be answered based on the provided context.
